KNN

Schedule with crontab

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.patches as mpatches
import seaborn as sb
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
plt.rcParams['figure.figsize'] = (16, 9)
plt.style.use('ggplot')
 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [2]:
dataframe = pd.read_csv(r"history.csv",sep=',')
#dataframe.head(10)
dataframe

,Model,Username,User_Country,User_Province,User_City,User_PostalCode,Numpublication,NumPublicationAdoption,NumPublicationHelp,NumPublicationStray,...,Housing,Other Pets,NumPets,Gender,Specie,Chip,Sex,Race,Age,State
0,User,admin,españa,Pontevedra,Desconocida,1,6,1,1,4,...,admin,dog,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Pages,admin,españa,Pontevedra,Desconocida,1,6,1,1,4,...,admin,dog,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Animal,admin,españa,Pontevedra,Desconocida,1,6,1,1,4,...,admin,dog,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Animal,admin,españa,Pontevedra,Desconocida,1,6,1,1,4,...,admin,dog,3,NaN,dog,no,unknow,caniche,4.0,unknown
4,PublicationAdoption,admin,españa,Pontevedra,Desconocida,1,6,1,1,4,...,admin,dog,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20003,PublicationAdoption,Timothy,Republica Dominicana,Segovia,Sestao,68046,4039,1551,1180,1308,...,quadplex,other,3,male,turtles,NaN,intact_female,Border Collie,10.0,sick
20004,PublicationAdoption,John,Colombia,Santa Cruz de Tenerife,Funes,90051,2771,224,1626,921,...,quadplex,turtles,11,nobinary,snake,yes,neutered_female,Slovensky Cuvac,10.0,healthy
20005,Event,Lloyd,Paraguay,Burgos,Galicia,38872,2457,324,133,2000,...,apartment,other,11,female,NaN,NaN,NaN,NaN,0.0,NaN
20006,PublicationStrayAddress,Mark,El Salvador,Alicante,Sanlúcar de Barrameda,72487,2469,1715,567,187,...,quadplex,bunny,0,male,NaN,NaN,NaN,NaN,0.0,NaN


In [3]:
my_cols = set(dataframe.columns)
# removing the desired column
my_cols.remove('NumPublicationStray')
my_cols.remove('NumPublicationHelp')
my_cols.remove('NumPublicationAdoption')
my_cols.remove('User_City')
my_cols.remove('User_Province')
my_cols.remove('User_PostalCode')
my_cols = list(my_cols)
df = dataframe[my_cols]
df["Gender"].fillna("Unknown", inplace = True)
df["Marital Status"].fillna("Unknown", inplace = True)
df["Housing"].fillna("Unknown", inplace = True)
df["Work"].fillna("Unknown", inplace = True)
df["Studies"].fillna("Unknown", inplace = True)
df["Other Pets"].fillna("Unknown", inplace = True)

df["Specie"].fillna("Unknown", inplace = True)
df["Chip"].fillna("Unknown", inplace = True)
df["Sex"].fillna("Unknown", inplace = True)
df["Race"].fillna("Unknown", inplace = True)
df["Age"].fillna(0, inplace = True)
df["State"].fillna("Unknown", inplace = True)
df["NumPets"].fillna("Unknown", inplace = True)
df["Children"].fillna(0, inplace = True)


In [4]:
df["Model"]  = np.where(df["Model"] == 'Animal' , 0, np.where(df["Model"] == 'PublicationHelp' , 1, np.where(df["Model"] == 'PublicationAdoption' , 2,np.where(df["Model"] == 'PublicationStray' , 3,np.where(df["Model"] == 'PublicationStrayAddress' , 4,np.where(df["Model"] == 'Event' , 5,np.where(df["Model"] == 'Comment' , 6,7)))))))
df['Chip'] = (df['Chip'] == 'yes').astype(int)
df["Marital Status"]  = np.where(df["Marital Status"] == 'single' , 0, np.where(df["Marital Status"] == 'married' , 1, np.where(df["Marital Status"] == 'divorced' , 2,np.where(df["Marital Status"] == 'separated' , 3, np.where(df["Marital Status"] == 'relationship' , 4, 5)))))
df["Sex"]  = np.where(df["Sex"] == 'intact_female' , 0, np.where(df["Sex"] == 'intact_male' , 1, np.where(df["Sex"] == 'neutered_female' , 2,np.where(df["Sex"] == 'castrated_male' , 3, np.where(df["Sex"] == 'unknow' , 4, 4)))))
df["Housing"]  = np.where(df["Housing"] == 'detached house' , 0, np.where(df["Housing"] == 'semi-detached house' , 1, np.where(df["Housing"] == 'terraced house' , 2,np.where(df["Housing"] == 'bungalows' , 3, np.where(df["Housing"] == 'studio' , 4, np.where(df["Housing"] == 'apartment' , 5, np.where(df["Housing"] == 'flat' , 6, np.where(df["Housing"] == 'attic' , 7,  np.where(df["Housing"] == 'ground floor' , 8, np.where(df["Housing"] == 'ground floor with garden' , 9, np.where(df["Housing"] == 'loft' , 10, np.where(df["Housing"] == 'duplex' , 11,  np.where(df["Housing"] == 'triplex' , 12, np.where(df["Housing"] == 'quadplex' , 14, 14))))))))))))))
df["Gender"]  = np.where(df["Gender"] == 'male' , 0, np.where(df["Gender"] == 'female' , 1, np.where(df["Gender"] == 'nobinary' , 2,3)))
df["Specie"]  = np.where(df["Specie"] == 'cat' , 0, np.where(df["Specie"] == 'dog' , 1, np.where(df["Specie"] == 'bunny' , 2,np.where(df["Specie"] == 'hamster' , 3,np.where(df["Specie"] == 'snake' , 4,np.where(df["Specie"] == 'turtles' , 5,6))))))
df["Other Pets"]  = np.where(df["Other Pets"] == 'cat' , 0, np.where(df["Other Pets"] == 'dog' , 1, np.where(df["Other Pets"] == 'bunny' , 2,np.where(df["Other Pets"] == 'hamster' , 3,np.where(df["Other Pets"] == 'snake' , 4,np.where(df["Other Pets"] == 'turtles' , 5,6))))))
df["State"]  = np.where(df["State"] == 'healthy' , 0, np.where(df["State"] == 'sick' , 1, np.where(df["State"] == 'adopted' , 2,np.where(df["State"] == 'dead' , 3,np.where(df["State"] == 'foster' , 4,np.where(df["State"] == 'other' , 5,6))))))
df

,Username,UserAge,Chip,Marital Status,Age,Housing,Children,Gender,NumComment,Studies,User_Country,Specie,Sex,State,Model,Race,Numpublication,NumPets,Work,Other Pets
0,admin,23,0,5,0.0,14,1,3,2,admin,españa,6,4,6,7,Unknown,6,3,admin,1
1,admin,23,0,5,0.0,14,1,3,2,admin,españa,6,4,6,7,Unknown,6,3,admin,1
2,admin,23,0,5,0.0,14,1,3,2,admin,españa,6,4,6,0,Unknown,6,3,admin,1
3,admin,23,0,5,4.0,14,1,3,2,admin,españa,1,4,6,0,caniche,6,3,admin,1
4,admin,23,0,5,0.0,14,1,3,2,admin,españa,6,4,6,2,Unknown,6,3,admin,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20003,Timothy,33,0,2,10.0,14,10,0,1785,Trabajo y atención social,Republica Dominicana,5,0,1,2,Border Collie,4039,3,Físico,6
20004,John,81,1,0,10.0,14,13,2,246,Administración y gestión de empresas,Colombia,4,2,0,2,Slovensky Cuvac,2771,11,Cerrajero,5
20005,Lloyd,3,0,0,0.0,5,15,1,273,Mercadotecnia y publicidad,Paraguay,6,4,6,5,Unknown,2457,11,Químico,6
20006,Mark,97,0,4,0.0,14,3,0,1930,Tecnologías de la información y comunicación,El Salvador,6,4,6,4,Unknown,2469,0,Archivólogo,2


In [6]:
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

import nltk
nltk.download('punkt')
nltk.download('stopwords')

ps = PorterStemmer()

preprocessedTextRace = []
preprocessedTextStudies = []
preprocessedTextCountry = []
preprocessedTextWork = []

for row in df.itertuples():
        text = word_tokenize(row[19]) ## indice de la columna que contiene el texto
        ## Remove stop words
        stops = set(stopwords.words("english"))
        text = [ps.stem(w) for w in text if not w in stops and w.isalnum()]
        text = " ".join(text)

        preprocessedTextWork.append(text)

preprocessedData = df
preprocessedData['processed_text_work'] = preprocessedTextWork

for row in df.itertuples():
        text = word_tokenize(row[10]) ## indice de la columna que contiene el texto
        ## Remove stop words
        stops = set(stopwords.words("english"))
        text = [ps.stem(w) for w in text if not w in stops and w.isalnum()]
        text = " ".join(text)

        preprocessedTextStudies.append(text)
        
preprocessedData = preprocessedData
preprocessedData['processed_text_studies'] = preprocessedTextStudies


for row in df.itertuples():

    text = word_tokenize(row[16]) ## indice de la columna que contiene el texto
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [ps.stem(w) for w in text if not w in stops and w.isalnum()]
    text = " ".join(text)
    
    preprocessedTextRace.append(text)

preprocessedData = preprocessedData
preprocessedData['processed_text_race'] = preprocessedTextRace

for row in df.itertuples():
        text = word_tokenize(row[11]) ## indice de la columna que contiene el texto
        ## Remove stop words
        stops = set(stopwords.words("english"))
        text = [ps.stem(w) for w in text if not w in stops and w.isalnum()]
        text = " ".join(text)

        preprocessedTextCountry.append(text)

preprocessedData = preprocessedData
preprocessedData['processed_text_country'] = preprocessedTextCountry


preprocessedData

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Username,UserAge,Chip,Marital Status,Age,Housing,Children,Gender,NumComment,Studies,...,Model,Race,Numpublication,NumPets,Work,Other Pets,processed_text_work,processed_text_studies,processed_text_race,processed_text_country
0,admin,23,0,5,0.0,14,1,3,2,admin,...,7,Unknown,6,3,admin,1,admin,admin,unknown,españa
1,admin,23,0,5,0.0,14,1,3,2,admin,...,7,Unknown,6,3,admin,1,admin,admin,unknown,españa
2,admin,23,0,5,0.0,14,1,3,2,admin,...,0,Unknown,6,3,admin,1,admin,admin,unknown,españa
3,admin,23,0,5,4.0,14,1,3,2,admin,...,0,caniche,6,3,admin,1,admin,admin,canich,españa
4,admin,23,0,5,0.0,14,1,3,2,admin,...,2,Unknown,6,3,admin,1,admin,admin,unknown,españa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20003,Timothy,33,0,2,10.0,14,10,0,1785,Trabajo y atención social,...,2,Border Collie,4039,3,Físico,6,físico,trabajo atención social,border colli,republica dominicana
20004,John,81,1,0,10.0,14,13,2,246,Administración y gestión de empresas,...,2,Slovensky Cuvac,2771,11,Cerrajero,5,cerrajero,administración gestión de empresa,slovenski cuvac,colombia
20005,Lloyd,3,0,0,0.0,5,15,1,273,Mercadotecnia y publicidad,...,5,Unknown,2457,11,Químico,6,químico,mercadotecnia publicidad,unknown,paraguay
20006,Mark,97,0,4,0.0,14,3,0,1930,Tecnologías de la información y comunicación,...,4,Unknown,2469,0,Archivólogo,2,archivólogo,tecnología de la información comunicación,unknown,el salvador


In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer

bagOfWordsModelWork = TfidfVectorizer()
bagOfWordsModelWork.fit(preprocessedData['processed_text_work'])
textsBoWWork = bagOfWordsModelWork.transform(preprocessedData['processed_text_work'])

bagOfWordsModelStudies = TfidfVectorizer()
bagOfWordsModelStudies.fit(preprocessedData['processed_text_studies'])
textsBoWStudies = bagOfWordsModelStudies.transform(preprocessedData['processed_text_studies'])

bagOfWordsModelRace = TfidfVectorizer()
bagOfWordsModelRace.fit(preprocessedData['processed_text_race'])
textsBoWRace= bagOfWordsModelRace.transform(preprocessedData['processed_text_race'])

bagOfWordsModelCountry = TfidfVectorizer()
bagOfWordsModelCountry.fit(preprocessedData['processed_text_country'])
textsBoWCountry= bagOfWordsModelCountry.transform(preprocessedData['processed_text_country'])
print("Finished")

Finished


In [33]:
textsBoWWork.shape

(20008, 42)

In [43]:
print(type(textsBoWWork))

<class 'scipy.sparse.csr.csr_matrix'>


In [35]:
bagOfWordsModelWork.get_feature_names()

['abogado',
 'admin',
 'archivólogo',
 'arquitecto',
 'artesano',
 'bibliotecólogo',
 'biólogo',
 'botánico',
 'carpintero',
 'cerrajero',
 'cirujano',
 'cocinero',
 'computista',
 'contador',
 'de',
 'electricista',
 'enfermero',
 'farmacólogo',
 'filólogo',
 'frutero',
 'físico',
 'geógrafo',
 'historiador',
 'ingeniero',
 'lavandero',
 'lechero',
 'matemático',
 'mecánico',
 'médico',
 'músico',
 'paleontólogo',
 'paramédico',
 'periodista',
 'politólogo',
 'profesor',
 'psicólogo',
 'químico',
 'secretaria',
 'sociólogo',
 'sonido',
 'técnico',
 'unknown']

Distancia

In [37]:
from sklearn.metrics import pairwise_distances

distance_matriWork= pairwise_distances(textsBoWWork,textsBoWWork ,metric='cosine')
distance_matriStudies= pairwise_distances(textsBoWStudies,textsBoWStudies ,metric='cosine')
distance_matriWork= pairwise_distances(textsBoWRace,textsBoWRace ,metric='cosine')
distance_matriWork= pairwise_distances(textsBoWCountry,textsBoWCountry ,metric='cosine')


Matriz a csv(preprocessedData)

In [42]:
a=np.array(textsBoWWork)
a
#preprocessedData['WorkVal'] = textsBoWWork
#preprocessedData['StudiesVal'] = textsBoWStudies
#preprocessedData['RaceVal'] = textsBoWRace
#preprocessedData['CountryVal'] = textsBoWCountry
#preprocessedData

array(<20008x42 sparse matrix of type '<class 'numpy.float64'>'
	with 21533 stored elements in Compressed Sparse Row format>, dtype=object)

In [31]:
print(distance_matriWork.shape)
print(type(distance_matriWork))

(20008, 20008)
<class 'numpy.ndarray'>


In [ ]:
from scipy import spatial
import sys
# To use item-based cosine similarity
if len(sys.argv) == 1:
    usuario = sys.argv[1]
else:
    print("Error - Introduce los argumentos correctamente")
sim_options = {
    "name": "cosine",
    "user_based": False,  # Compute  similarities between items
}
spatial.distance.euclidean(c, a)